In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the tokenizer and model
model_path = "/home/sjoshi/lmm/lm-train/checkpoints/tinystories/checkpoint-33121"  # Replace with the path where your model is saved
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Define a function to generate text
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Generate text using the model
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

/home/sjoshi/anaconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Example usage
prompt = "Once upon a time, there was a little boy named Billy"
generated_text = generate_text(prompt, max_length=200)
print(generated_text)

Once upon a time, there was a little boy named Billy. He was three years old and loved to explore. One day, he decided to go on an adventure.

Billy walked through the forest and soon he came across a big, scary cave. It was dark and scary, but Billy was brave. So he stepped inside. 
  He saw a lot of strange things. There were big rocks and strange creatures. Billy wanted to touch them, so he reached out his hand. Suddenly, the cave started to shake and a loud noise filled the air. The noise was so loud that it scared Billy and he ran away. But he was too scared to move. Then he saw something else. A big and hairy monster was standing in front of him!  

 Billy screamed and ran as fast as he could. When he got home, his mom was there to help him. She hugged him and told him that he should never go into the scary forest again.
